# Objective

## This notebook aims to analyze how Kagglers from your Country differ from the average Kaggler

<b>Data Source</b>: kaggle_survey_2020_responses.csv

"Copy and Edit" this notebook to choose your own country and other parameters, and watch how the graphs and insights change accordingly!

<b>Please don't forget to <font color=green>UPVOTE</font> if you find this interesting. Feedback is more than welcome :)</b>

### This analysis convers <font color=green>ALL</font> the questions asked in the Kaggle 2020 DS and ML survey, divided into the below sections:
* Demographics
* Personal Background
* Personal Preferences
* Learning Preferences
* Workplace

### So hold on tight, this is going to take a while!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('mode.chained_assignment', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv', skiprows=1)
df[df.columns[3]].replace({'United Kingdom of Great Britain and Northern Ireland':'UK',
                           'Iran, Islamic Republic of...':'Iran',
                           'United Arab Emirates':'UAE',
                           'United States of America':'USA',
                           'Viet Nam':'Vietnam'}, inplace=True)

print(f"List of countries:\n{np.sort(df[df.columns[3]].unique())}\n")

In [ ]:
# Select a country from the list above
country = 'India'

In [ ]:
if country not in df[df.columns[3]].unique():
    raise ValueError(f'{country} not found in the list')
df['country_agg'] = np.where(df[df.columns[3]]==country,country,'Others')

# Demographics
## 1. Country

In [ ]:
fig = px.pie(df, df.columns[3], 
       title=f"{len(df[df[df.columns[3]]==country])*100/len(df):.2f}% of all survey respondents are from {country}", 
       hole=0.6)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

## 2. Age

In [ ]:
fig = px.pie(df, df.columns[1], title='56% of all Kagglers are less than 30 years old', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

age_pct = len(df[(df[df.columns[3]]==country) & (df[df.columns[1]].isin(['18-21','22-24','25-29']))])*100/len(df[df[df.columns[3]]==country])
if age_pct < 56:
    title = f"{country} is older, with {age_pct:.0f}% of Kagglers being under under 30"
elif age_pct > 56:
    title = f"{country} is younger, with {age_pct:.0f}% of Kagglers being under under 30"
else:
    title = f"{age_pct:.0f}% of Kagglers from {country} are also under 30"
fig = px.pie(df[df[df.columns[3]]==country], df.columns[1], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df['age1'] = df.iloc[:,1].str.split('-').str[0]
df['age1'].replace('70+','70', inplace=True)
df['age2'] = df.iloc[:,1].str.split('-').str[1]
df['age1'] = df.age1.astype('int')
df.age2.fillna(70, inplace=True)
df['age2'] = df.age2.astype('int')
df['age'] = (df.age1+df.age2)/2

global_median = df.age.mean()
country_median = df[df.country_agg==country].age.mean()

if country_median <= global_median:
    title = f"With an average age of {country_median:.0f},<br>Kagglers from {country} are generally {global_median - country_median:.0f} years younger than the average Kaggler"
else:
    title = f"With an average age of {country_median:.0f},<br>Kagglers from {country} are generally {country_median - global_median:.0f} years younger than the average Kaggler"

loc = df.groupby(df.columns[3]).age.mean().sort_values(ascending=False).index.to_list().index(country)
color = ['#636EFA']*len(df.groupby(df.columns[3]).age.mean().sort_values(ascending=False).index)
color[loc] = 'orange'

fig = go.Figure(data=[go.Bar(x=df.groupby(df.columns[3]).age.mean().sort_values(ascending=False).index
       , y=df.groupby(df.columns[3]).age.mean().sort_values(ascending=False)
            , marker_color=color)])

fig.update_layout(
    shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= global_median, y1= global_median,
      xref= 'x', x0= -0.5, x1= len(df.groupby(df.columns[3]).age)-0.5
    )],
    title=title,
    xaxis_title=None,
    yaxis_title='Age')

fig.add_annotation(x=len(df.groupby(df.columns[3]).age)*0.95, y=global_median, xshift=-20, yshift=10,
            text="Global Average",
            showarrow=False)
fig.show()

df_country = df[df.country_agg==country][df.columns[1]].value_counts(normalize=True).sort_index()
df_others = df[df.country_agg=='Others'][df.columns[1]].value_counts(normalize=True).sort_index()
for index in df_others.index:
    if index not in df_country.index:
        df_country[index] = 0
df_country.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, y=df_country.values*100),
    go.Bar(name='Others', y=df_others.values*100)
])

# Change the bar mode
fig.update_layout(
    barmode='group',
    title=f'Age distribution of Kagglers from {country} compared to others',
    xaxis_title='Age',
    yaxis_title='Percentage of respondents',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(12)],
        ticktext = df[df.columns[1]].sort_values().unique()
    )
)

fig.show()

## 3. Gender diversity

In [ ]:
fig = px.pie(df, df.columns[2], title=f"Gender distribution of all Kagglers", hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

categories = sorted(df.iloc[:,2].unique())
df_country_gender = df[df.country_agg==country].iloc[:,2].value_counts(normalize=True).sort_index()
df_other_platform = df[df.country_agg=='Others'].iloc[:,2].value_counts(normalize=True).sort_index()

fig = go.Figure(data=[
    go.Bar(name=country, x=categories, y=df_country_gender.values*100),
    go.Bar(name='Others', x=categories, y=df_other_platform.values*100)
])
# Change the bar mode
fig.update_layout(
    title=f'Gender of respondents from {country} compared to rest of the world',
    xaxis_title=None,
    yaxis_title='Percentage',
    xaxis={'categoryorder':'array',
           'categoryarray':categories}
)
fig.show()

genders = [x for x in df.iloc[:,2].unique() if x !='Prefer not to say']

for gender in genders:
    df_all = df.groupby(df.columns[3])[df.columns[2]].value_counts().groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))[:,gender].sort_values(ascending=False)
    if country in df_all.index:
        country_avg = df_all[country]
        global_avg = len(df[df[df.columns[2]]==gender])*100/len(df)
        gndr_pct = len(df[df[df.columns[2]]==gender])*100/len(df)
        title=f"{country_avg:.1f}% of Kagglers from {country} identify as \"{gender}\",<br>compared to the global average of {global_avg:.1f}%"

        loc = df_all.index.to_list().index(country)
        color = ['#636EFA']*len(df_all.index)
        color[loc] = 'orange'

        fig = go.Figure(data=[go.Bar(x=df_all.index, y=df_all.values, marker_color=color)])
        fig.update_layout(
            shapes=[
                dict(
                  type= 'line',
                  yref= 'y', y0= global_avg, y1= global_avg,
                  xref= 'x', x0= -0.5, x1= len(df_all.index)-0.5
                )],
            title=title,
            xaxis_title=None,
            yaxis_title='Percentage')
        fig.add_annotation(x=len(df_all.index)*0.95, y=global_avg, xshift=-20, yshift=10,
                    text="Global Average",
                    showarrow=False)
        fig.show()

    else:
        print(f"Nobody from {country} identified as {gender}")

# Personal Background
## 4. Academic qualification
<font color=red> Only includes respondents who answered the question "<b>What is the highest level of formal education that you have attained or plan to attain within the next 2 years?</b>" </font> 

In [ ]:
df_academic = df[(df[df.columns[4]]!='I prefer not to answer') & (~df[df.columns[4]].isna())]

fig = px.pie(df_academic, df_academic.columns[4], 
             title="Surprisingly, globally more Kagglers reported having a Master's degree (41%)<br>than a Bachelor's (36%)", hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

most_comm = df_academic[df_academic.iloc[:,3]==country].iloc[:,4].value_counts(normalize=True)[[0]]
if most_comm.index[0] == "Master's degree":
    title = f"The same trend is observed in {country},<br>with {most_comm[0]*100:.0f}% respondents reporting having a Master's degree"
else:
    title = f"However, in the case of {country},<br>more respondents reported having a {most_comm.index[0]} at {most_comm[0]*100:.0f}%"
    
fig = px.pie(df_academic[df_academic.iloc[:,3]==country], df_academic.columns[4], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

categories = ['No formal education past high school',
             'Some college/university study without earning a bachelor’s degree',
             'Professional degree',
             'Bachelor’s degree',
             'Master’s degree',
             'Doctoral degree']

df_country_agg = df[df.country_agg==country].iloc[:,4].value_counts(normalize=True)
df_country_agg.index = pd.Categorical(df_country_agg.index, categories)
df_country_agg.sort_index(inplace=True)

df_others_agg = df[df.country_agg=='Others'].iloc[:,4].value_counts(normalize=True)
df_others_agg.index = pd.Categorical(df_others_agg.index, categories)
df_others_agg.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, x=categories, y=df_country_agg.values*100),
    go.Bar(name='Others', x=categories, y=df_others_agg.values*100)
])
# Change the bar mode
fig.update_layout(
    title=f'Academic Qualification of respondents from {country} compared to Other countries',
    xaxis_title=None,
    yaxis_title='Percentage'
)
fig.show()

qualifications = df_academic.iloc[:,4].unique()

for qualification in qualifications:
    df_all = df.groupby(df.columns[3])[df.columns[4]].value_counts().groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))[:,qualification].sort_values(ascending=False)
    country_avg = df_all[country]
    global_avg = len(df[df[df.columns[4]]==qualification])*100/len(df)
    gndr_pct = len(df[df[df.columns[4]]==qualification])*100/len(df)
    title=f"{country_avg:.1f}% of Kagglers from {country} have {qualification},<br>compared to the global average of {global_avg:.1f}%"

    loc = df_all.index.to_list().index(country)
    color = ['#636EFA']*len(df_all.index)
    color[loc] = 'orange'

    fig = go.Figure(data=[go.Bar(x=df_all.index, y=df_all.values, marker_color=color)])
    fig.update_layout(
        shapes=[
            dict(
              type= 'line',
              yref= 'y', y0= global_avg, y1= global_avg,
              xref= 'x', x0= -0.5, x1= len(df_all.index)-0.5
            )],
        title=title,
        xaxis_title=None,
        yaxis_title='Percentage')
    fig.add_annotation(x=len(df_all.index)*0.95, y=global_avg, xshift=-20, yshift=10,
                text="Global Average",
                showarrow=False)
    fig.show()

## 5. Job title
<font color=red> Only includes respondents who answered the question "<b>Select the title most similar to your current role (or most recent title if retired)</b>" </font>

In [ ]:
df_job = df[~df[df.columns[5]].isna()]
fig = px.pie(df_job, df_job.columns[5], title='Globally, 27% Kagglers are students, followed by 14% being Data Scientists', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

df_country_job = df_job[df_job.iloc[:,3]==country].iloc[:,5].value_counts(normalize=True)
com_job = df_country_job[[0]].index[0]
if com_job=='Student':
    title = f"For {country} too, most Kagglers are {com_job}s ({df_country_job[0]*100:.0f}%), followed by {df_country_job[[1]].index[0]}s ({df_country_job[1]*100:.0f}%)"
else:
    title = f"However, for {country}, most Kagglers are {com_job}s ({df_country_job[0]*100:.0f}%), followed by {df_country_job[[1]].index[0]}s ({df_country_job[1]*100:.0f}%)"

    
fig = px.pie(df_job[df_job.iloc[:,3]==country], df_job.columns[5], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

jobs = df_job.iloc[:,5].unique()
for job in jobs:
    df_all = df.groupby(df.columns[3])[df.columns[5]].value_counts().groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))[:,job].sort_values(ascending=False)
    country_avg = df_all[country]
    global_avg = len(df[df[df.columns[5]]==job])*100/len(df)

    loc = df_all.index.to_list().index(country)
    color = ['#636EFA']*len(df_all.index)
    color[loc] = 'orange'

    fig = go.Figure(data=[go.Bar(x=df_all.index, y=df_all.values, marker_color=color)])

    fig.update_layout(
        shapes=[
        dict(
          type= 'line',
          yref= 'y', y0= global_avg, y1= global_avg,
          xref= 'x', x0= -0.5, x1= len(df_all.index)-0.5
        )],
        title=f"{country_avg:.1f}% of Kagglers from {country} reported their job-title as \'{job}\',<br>compared to the global average of {global_avg:.1f}%",
        xaxis_title=None,
        yaxis_title='Percentage',)

    fig.add_annotation(x=len(df_all.index), y=global_avg, xshift=-60, yshift=10,
                text="Global Average",
                showarrow=False)

    fig.show()

## 6. Annual Compensation (in USD)
<font color=red>Only includes respondents who are <b>employed</b> and have answered the question '<b>What is your current yearly compensation?</b>'</font>

In [ ]:
df_comp = df[(~df.iloc[:,118].isna()) & (~df.iloc[:,5].isin(['Student','Currently not employed']))]

df_comp['comp1'] = df_comp.iloc[:,118].str.split('-').str[0].apply(lambda x: x.replace(',','').replace('$','').replace('>','')).astype('int')
df_comp['comp2'] = df_comp.iloc[:,118].str.split('-').str[1].fillna('500000').apply(lambda x: x.replace(',','')).astype('int').astype('int')
df_comp['comp'] = (df_comp.comp1+df_comp.comp2)/2

global_median = df_comp.comp.mean()
country_median = df_comp[df_comp.country_agg==country].comp.mean()

if country_median <= global_median:
    title = f"With an average annual compensation of {country_median:.0f} USD,<br>Kagglers from {country} generally earn less than the global average ({global_median:.0f} USD)"
else:
    title = f"With an average annual compensation of {country_median:.0f} USD,<br>Kagglers from {country} generally earn more than the global average ({global_median:.0f} USD)"

loc = df_comp.groupby(df_comp.columns[3]).comp.mean().sort_values(ascending=False).index.to_list().index(country)
color = ['#636EFA']*len(df_comp.groupby(df_comp.columns[3]).comp.mean().sort_values(ascending=False).index)
color[loc] = 'orange'

fig = go.Figure(data=[go.Bar(x=df_comp.groupby(df_comp.columns[3]).comp.mean().sort_values(ascending=False).index
       , y=df_comp.groupby(df.columns[3]).comp.mean().sort_values(ascending=False)
            , marker_color=color)])

fig.update_layout(
    shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= global_median, y1= global_median,
      xref= 'x', x0= -0.5, x1= len(df_comp.groupby(df_comp.columns[3]).comp)-0.5
    )],
    title=title,
    xaxis_title=None,
    yaxis_title='Average Annual Compensation (in USD)')

fig.add_annotation(x=len(df_comp.groupby(df.columns[3]).comp)*0.95, y=global_median, xshift=-20, yshift=10,
            text="Global Average",
            showarrow=False)
fig.show()

most_common = df_comp[df_comp.country_agg==country].groupby(df_comp.columns[118]).size().sort_values(ascending=False)
most_common_comp = most_common.index[0]
most_common_comp_pct = most_common[0]*100/most_common.sum()

fig = px.pie(df_comp[df_comp.country_agg==country], df_comp.columns[118], 
             title=f'{most_common_comp_pct:.0f}% of Kagglers from {country} reported an annual compensation between {most_common_comp}', 
             hole=0.6)
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

## 7. Coding Experience
<font color=red> Only includes respondents who answered the question "<b>For how many years have you been writing code and/or programming?</b>" </font>

In [ ]:
df_coding = df[~df[df.columns[6]].isna()]

fig = px.pie(df_coding, df_coding.columns[6], title='65% of all survey respondents have been coding for less than 5 years', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

country_coding_exp = len(df_coding[(df_coding[df_coding.columns[3]]==country) & (df_coding[df_coding.columns[6]].isin(['3-5 years','< 1 years','1-2 years']))])*100/len(df_coding[df_coding[df_coding.columns[3]]==country])
if country_coding_exp > 65:
    title = f"For {country}, this percentage increases to {country_coding_exp:.0f}%"
elif country_coding_exp < 65:
    title = f"For {country}, this percentage decreases to {country_coding_exp:.0f}%" 
else:
    title = f"At {country_coding_exp:.0f}%, it is the same for {country} too<br>The average Kaggler from {country} about the same coding experience as the global average"
fig = px.pie(df_coding[df_coding[df_coding.columns[3]]==country], df_coding.columns[6], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df_coding[df_coding.columns[6]] = df_coding[df_coding.columns[6]].replace('< 1 years','0-1 years').replace('I have never written code','0-0').apply(lambda x: x.split()[0])
df_coding['code1'] = df_coding.iloc[:,6].str.split('-').str[0].replace('20+','20').astype('int')
df_coding['code2'] = df_coding.iloc[:,6].str.split('-').str[1].fillna('20').astype('int')
df_coding.groupby([df_coding.columns[6]]+['code1','code2']).size()
df_coding['code'] = (df_coding.code1+df_coding.code2)/2

global_median = df_coding.code.mean()
country_median = df_coding[df_coding.country_agg==country].code.mean()

if country_median <= global_median:
    title = f"The average Kaggler from {country} has been coding for {country_median:.1f} years,<br>less than the global average of {global_median:.1f} years"
else:
    title = f"The average Kaggler from {country} has been coding for {country_median:.1f} years,<br>more than the global average of {global_median:.1f} years"

loc = df_coding.groupby(df_coding.columns[3]).code.mean().sort_values(ascending=False).index.to_list().index(country)
color = ['#636EFA']*len(df_coding.groupby(df_coding.columns[3]).code.mean().sort_values(ascending=False).index)
color[loc] = 'orange'

fig = go.Figure(data=[go.Bar(x=df_coding.groupby(df_comp.columns[3]).code.mean().sort_values(ascending=False).index
       , y=df_coding.groupby(df_coding.columns[3]).code.mean().sort_values(ascending=False)
            , marker_color=color)])

fig.update_layout(
    shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= global_median, y1= global_median,
      xref= 'x', x0= -0.5, x1= len(df_coding.groupby(df_coding.columns[3]).code)-0.5
    )],
    title=title,
    xaxis_title=None,
    yaxis_title='Average coding experience')

fig.add_annotation(x=len(df_coding.groupby(df.columns[3]).code)*0.95, y=global_median, xshift=-20, yshift=10,
            text="Global Average",
            showarrow=False)
fig.show()

categories = ['I have never written code','< 1 years','1-2 years','3-5 years','5-10 years','10-20 years','20+ years']

df_country_agg = df[df.country_agg==country].iloc[:,6].value_counts(normalize=True)
df_country_agg.index = pd.Categorical(df_country_agg.index, categories)
df_country_agg.sort_index(inplace=True)

df_others_agg = df[df.country_agg=='Others'].iloc[:,6].value_counts(normalize=True)
df_others_agg.index = pd.Categorical(df_others_agg.index, categories)
df_others_agg.sort_index(inplace=True)

for index in df_others_agg.index:
        if index not in df_country_agg.index:
            df_country_agg[index] = 0
df_country_agg.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, x=categories, y=df_country_agg.values*100),
    go.Bar(name='Others', x=categories, y=df_others_agg.values*100)
])
# Change the bar mode
fig.update_layout(
    title=f'Coding Experience of respondents from {country} compared to Other countries',
    xaxis_title=None,
    yaxis_title='Percentage'
)
fig.show()

# Personal Preferences
## 8. Programming language
<font color=red>Only includes respondents who chose atleast one option for the question "<b>What programming languages do you use on a regular basis?</b>"</font>

In [ ]:
programming_cols = [col for col in df.columns 
                    if 'What programming languages do you use on a regular basis?' in col]
df_programming = df.copy()
mapper = [col.split('-')[-1].lstrip() for col in programming_cols]
mapping_dict = dict(zip(programming_cols,mapper))
df_programming = df_programming[programming_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_programming.dropna(how='all', subset=mapper, inplace=True)
df_programming.drop(columns=["None"], inplace=True)

fig = px.bar(df_programming[df_programming.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'87% of all respondents use Python on a regular basis',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_programming[df_programming.iloc[:,-1]==country][df_programming.columns[:-2]].count().sort_values(ascending=False)
pop_lang = country_most_pop.index[0]
most_pop_pct = country_most_pop[0]*100/len(df_programming[df_programming.iloc[:,-1]==country])
if pop_lang=='Python':
    if most_pop_pct > 87:
        title = f'{pop_lang} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_lang} remains the most popular programming language in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_lang} is more popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_programming[df_programming.iloc[:,-1]==country][df_programming.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 9. IDE
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following integrated development environments (IDE's) do you use on a regular basis?</b>'</font>

In [ ]:
ide_cols = [col for col in df.columns 
            if "Which of the following integrated development environments (IDE's) do you use on a regular basis?" in col]
df_ide = df.copy()
mapper = [col.split('-')[-1].lstrip() for col in ide_cols]
mapping_dict = dict(zip(ide_cols,mapper))
df_ide = df_ide[ide_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_ide.dropna(how='all', subset=mapper, inplace=True)
df_ide.drop(columns=['Click to write Choice 13','None'], inplace=True)
df_ide.rename(columns={'Jupyter (JupyterLab, Jupyter Notebooks, etc) ':'Jupyter'}, inplace=True)

fig = px.bar(df_ide[df_ide.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'64% of all respondents use Jupyter',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_ide[df_ide.iloc[:,-1]==country][df_ide.columns[:-2]].count().sort_values(ascending=False)
pop_ide = country_most_pop.index[0]
most_pop_pct = country_most_pop[0]*100/len(df_ide[df_ide.iloc[:,-1]==country])
if pop_ide=='Jupyter':
    if most_pop_pct > 64:
        title = f'{pop_ide} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_ide} remains the most popular IDE in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_ide} is more,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_ide[df_ide.iloc[:,-1]==country][df_ide.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 10. Hosted Notebooks
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following hosted notebook products do you use on a regular basis?</b>'</font>

In [ ]:
nb_cols = [col for col in df.columns 
           if "Which of the following hosted notebook products do you use on a regular basis?" in col]
df_nb = df.copy()
mapper = [col.split('-')[-1].lstrip() for col in nb_cols]
mapping_dict = dict(zip(nb_cols,mapper))
df_nb = df_nb[nb_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_nb.dropna(how='all', subset=mapper, inplace=True)
df_nb.drop(columns=['None'], inplace=True)

fig = px.bar(df_nb[df_nb.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'37% of all respondents use Colab Notebooks',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_nb[df_nb.iloc[:,-1]==country][df_nb.columns[:-2]].count().sort_values(ascending=False)
pop_nb = country_most_pop.index[0]
most_pop_pct = country_most_pop[0]*100/len(df_nb[df_nb.iloc[:,-1]==country])
if pop_nb=='Colab Notebooks':
    if most_pop_pct > 37:
        title = f'{pop_nb} are even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis'
    else:
        title = f'{pop_nb} remain the most popular hosted notebooks in {country} too,<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis'
else:
    title = f"However, in {country}, {pop_nb} are more popular,<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis"

fig = px.bar(df_nb[df_nb.iloc[:,-1]==country][df_nb.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 11. Computing Platform

<font color=red>Only includes respondents who answered the question '<b>What type of computing platform do you use most often for your data science projects?</b>'</font>

In [ ]:
df_platform = df[(~df.iloc[:,47].isna()) & (df.iloc[:,47]!='None')]

fig = px.pie(df_platform, df_platform.columns[47], 
             title=f"80% of all survey respondents use a Personal Computer/Laptop<br>as their primary computing platform for DS projects", hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

df_country_platform = df_platform[df_platform.iloc[:,3]==country][df.columns[47]].value_counts(normalize=True)*100

country_avg = df_country_platform.loc['A personal computer or laptop']

if country_avg > 80:
    title=f"In {country}, this percentage increases to {country_avg:.0f}%"
else:
    title=f"In {country}, this percentage decreases to {country_avg:.0f}%"
    
fig = px.pie(df_country_platform, df_country_platform.index, df_country_platform.values, title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

categories = ['A cloud computing platform', 'A deep learning workstation', 'A personal computer or laptop', 'Other']
df_country_platform = df_platform[df_platform.country_agg==country].iloc[:,47].value_counts(normalize=True).sort_index()
df_other_platform = df_platform[df_platform.country_agg=='Others'].iloc[:,47].value_counts(normalize=True).sort_index()

for index in df_other_platform.index:
        if index not in df_country_platform.index:
            df_country_platform[index] = 0
df_country_platform.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, x=categories, y=df_country_platform.values*100),
    go.Bar(name='Others', x=categories, y=df_other_platform.values*100)
])
# Change the bar mode
fig.update_layout(
    title=f'Coding Platform preference of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage',
    xaxis={'categoryorder':'array',
           'categoryarray':categories}
)
fig.show()

## 12. Data Visualization Library
<font color=red>Only includes respondents who chose atleast one option for the question '<b>What data visualization libraries or tools do you use on a regular basis?</b>'</font>

In [ ]:
viz_cols = [col for col in df.columns 
           if "What data visualization libraries or tools do you use on a regular basis?" in col]
df_viz = df.copy()
mapper = [col.split('-')[-1].strip() for col in viz_cols]
mapping_dict = dict(zip(viz_cols,mapper))
df_viz = df_viz[viz_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_viz.dropna(how='all', subset=mapper, inplace=True)
df_viz.drop(columns=['None'], inplace=True)

fig = px.bar(df_viz[df_viz.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'75% of all respondents use Matplotlib',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_viz[df_viz.iloc[:,-1]==country][df_viz.columns[:-2]].count().sort_values(ascending=False)
pop_viz = country_most_pop.index[0]
most_pop_pct = country_most_pop[0]*100/len(df_viz[df_viz.iloc[:,-1]==country])
if pop_viz=='Matplotlib':
    if most_pop_pct > 75:
        title = f'{pop_viz} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_viz} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_viz} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_viz[df_viz.iloc[:,-1]==country][df_viz.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 13. Machine Learning Frameworks
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following machine learning frameworks do you use on a regular basis?</b>'</font>

In [ ]:
ml_cols = [col for col in df.columns 
           if "Which of the following machine learning frameworks do you use on a regular basis?" in col]
df_ml = df.copy()
mapper = [col.split('-')[-1].strip() for col in ml_cols]
mapper[0]='Scikit-learn'
mapping_dict = dict(zip(ml_cols,mapper))
df_ml = df_ml[ml_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_ml.dropna(how='all', subset=mapper, inplace=True)
df_ml.drop(columns=['None'], inplace=True)

fig = px.bar(df_ml[df_ml.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'72% of all respondents use Scikit-learn',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_ml[df_ml.iloc[:,-1]==country][df_ml.columns[:-2]].count().sort_values(ascending=False)
pop_ml = country_most_pop.index[0]
most_pop_pct = country_most_pop[0]*100/len(df_ml[df_ml.iloc[:,-1]==country])
if pop_ml=='Scikit-learn':
    if most_pop_pct > 72:
        title = f'{pop_ml} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_ml} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_ml} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_ml[df_ml.iloc[:,-1]==country][df_ml.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 14. Machine Learning Algorithms
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following ML algorithms do you use on a regular basis?</b>'</font>

In [ ]:
ml_cols = [col for col in df.columns 
           if "Which of the following ML algorithms do you use on a regular basis?" in col]
df_ml = df.copy()
mapper = [col.split('-')[-1].strip() for col in ml_cols]
mapper[9]='Transformer Networks (BERT, gpt-3, etc)'
mapping_dict = dict(zip(ml_cols,mapper))
df_ml = df_ml[ml_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_ml.dropna(how='all', subset=mapper, inplace=True)
df_ml.drop(columns=['None'], inplace=True)

fig = px.bar(df_ml[df_ml.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'76% of all respondents use Linear or Logistic Regression',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_ml[df_ml.iloc[:,-1]==country][df_ml.columns[:-2]].count().sort_values(ascending=False)
pop_ml = country_most_pop.index[0]
most_pop_pct = country_most_pop[0]*100/len(df_ml[df_ml.iloc[:,-1]==country])
if pop_ml=='Linear or Logistic Regression':
    if most_pop_pct > 76:
        title = f'{pop_ml} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_ml} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_ml} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_ml[df_ml.iloc[:,-1]==country][df_ml.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 15. Computer Vision Methods
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which categories of computer vision methods do you use on a regular basis?</b>'</font>

In [ ]:
cv_cols = [col for col in df.columns 
           if "Which categories of computer vision methods do you use on a regular basis?" in col]
df_cv = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cv_cols]
mapping_dict = dict(zip(cv_cols,mapper))
df_cv = df_cv[cv_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_cv.dropna(how='all', subset=mapper, inplace=True)
df_cv.drop(columns=['None'], inplace=True)

fig = px.bar(df_cv[df_cv.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'59% of all respondents use Image classification and other general purpose networks',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_cv[df_cv.iloc[:,-1]==country][df_cv.columns[:-2]].count().sort_values(ascending=False)
pop_cv = country_most_pop.index[0]
if '(' in pop_cv:
    pop_cv = pop_cv.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_cv[df_cv.iloc[:,-1]==country])
if pop_cv=='Image classification and other general purpose networks':
    if most_pop_pct > 59:
        title = f'{pop_cv} are even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis'
    else:
        title = f'{pop_cv} remain the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis'
else:
    title = f"However, in {country}, {pop_cv} are popular,<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis"

fig = px.bar(df_cv[df_cv.iloc[:,-1]==country][df_cv.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 16. NLP Methods
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following natural language processing (NLP) methods do you use on a regular basis?</b>'</font>

In [ ]:
nlp_cols = [col for col in df.columns 
           if "Which of the following natural language processing (NLP) methods do you use on a regular basis?" in col]
df_nlp = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in nlp_cols]
mapping_dict = dict(zip(nlp_cols,mapper))
df_nlp = df_nlp[nlp_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_nlp.dropna(how='all', subset=mapper, inplace=True)
df_nlp.drop(columns=['None'], inplace=True)

fig = px.bar(df_nlp[df_nlp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'55% of all respondents use Word embeddings/vectors',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_nlp[df_nlp.iloc[:,-1]==country][df_nlp.columns[:-2]].count().sort_values(ascending=False)
pop_nlp = country_most_pop.index[0]
if '(' in pop_nlp:
    pop_nlp = pop_nlp.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_nlp[df_nlp.iloc[:,-1]==country])
if pop_nlp.strip()=='Word embeddings/vectors':
    if most_pop_pct > 55:
        title = f'{pop_nlp} are even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis'
    else:
        title = f'{pop_nlp} remain the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis'
else:
    title = f"However, in {country}, {pop_nlp} are popular,<br>with {most_pop_pct:.0f}% of respondents using them on a regular basis"

fig = px.bar(df_nlp[df_nlp.iloc[:,-1]==country][df_nlp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 17. Cloud Computing Platforms
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following cloud computing platforms do you use on a regular basis?</b>'</font>

In [ ]:
ccp_cols = [col for col in df.columns 
           if "Which of the following cloud computing platforms do you use on a regular basis?" in col]
df_ccp = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in ccp_cols]
mapping_dict = dict(zip(ccp_cols, mapper))
df_ccp = df_ccp[ccp_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_ccp.dropna(how='all', subset=mapper, inplace=True)
df_ccp.drop(columns=['None'], inplace=True)

fig = px.bar(df_ccp[df_ccp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'41% of all respondents use AWS',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_ccp[df_ccp.iloc[:,-1]==country][df_ccp.columns[:-2]].count().sort_values(ascending=False)
pop_ccp = country_most_pop.index[0].strip()
if '(' in pop_nlp:
    pop_ccp = pop_ccp.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_ccp[df_ccp.iloc[:,-1]==country])
if pop_ccp == 'Amazon Web Services (AWS)':
    if most_pop_pct > 41:
        title = f'{pop_ccp} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_ccp} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_ccp} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_ccp[df_ccp.iloc[:,-1]==country][df_ccp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 18. Cloud Computing Products
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Do you use any of the following cloud computing products on a regular basis?</b>'</font>

In [ ]:
ccp_cols = [col for col in df.columns 
           if "Do you use any of the following cloud computing products on a regular basis?" in col]
df_ccp = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in ccp_cols]
mapping_dict = dict(zip(ccp_cols, mapper))
df_ccp = df_ccp[ccp_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_ccp.dropna(how='all', subset=mapper, inplace=True)
df_ccp.drop(columns=['No / None'], inplace=True)

fig = px.bar(df_ccp[df_ccp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'36% of all respondents use Amazon EC2',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_ccp[df_ccp.iloc[:,-1]==country][df_ccp.columns[:-2]].count().sort_values(ascending=False)
pop_ccp = country_most_pop.index[0].strip()
if '(' in pop_nlp:
    pop_ccp = pop_ccp.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_ccp[df_ccp.iloc[:,-1]==country])
if pop_ccp == 'Amazon EC2':
    if most_pop_pct > 36:
        title = f'{pop_ccp} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_ccp} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_ccp} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_ccp[df_ccp.iloc[:,-1]==country][df_ccp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 19. Machine Learning Products
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Do you use any of the following machine learning products on a regular basis?</b>'</font>

In [ ]:
mlp_cols = [col for col in df.columns 
           if "Do you use any of the following machine learning products on a regular basis?" in col]
df_mlp = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in mlp_cols]
mapping_dict = dict(zip(mlp_cols, mapper))
df_mlp = df_mlp[mlp_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_mlp.dropna(how='all', subset=mapper, inplace=True)
df_mlp.drop(columns=['No / None'], inplace=True)

fig = px.bar(df_mlp[df_mlp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'16% of all respondents use Google Cloud AI Platform / Google Cloud ML Engine',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_mlp[df_mlp.iloc[:,-1]==country][df_mlp.columns[:-2]].count().sort_values(ascending=False)
pop_mlp = country_most_pop.index[0].strip()
if '(' in pop_mlp:
    pop_mlp = pop_mlp.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_mlp[df_mlp.iloc[:,-1]==country])
if pop_mlp == 'Google Cloud AI Platform / Google Cloud ML Engine':
    if most_pop_pct > 16:
        title = f'{pop_mlp} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_mlp} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_mlp} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_mlp[df_mlp.iloc[:,-1]==country][df_mlp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 20. Big Data Products
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis?</b>'</font>

In [ ]:
bdp_cols = [col for col in df.columns 
           if "Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis?" in col]
df_bdp = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in bdp_cols]
mapping_dict = dict(zip(bdp_cols, mapper))
df_bdp = df_bdp[bdp_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_bdp.dropna(how='all', subset=mapper, inplace=True)
df_bdp.drop(columns=['None'], inplace=True)

fig = px.bar(df_bdp[df_bdp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'37% of all respondents use MySQL',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_bdp[df_bdp.iloc[:,-1]==country][df_bdp.columns[:-2]].count().sort_values(ascending=False)
pop_bdp = country_most_pop.index[0].strip()
if '(' in pop_bdp:
    pop_bdp = pop_bdp.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_bdp[df_bdp.iloc[:,-1]==country])
if pop_bdp == 'MySQL':
    if most_pop_pct > 37:
        title = f'{pop_bdp} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_bdp} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_bdp} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_bdp[df_bdp.iloc[:,-1]==country][df_bdp.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()


df_bdp1 = df[(~df.iloc[:,173].isna()) & (df.iloc[:,173]!='None')]

fig = px.pie(df_bdp1, df_bdp1.columns[173], 
             title=f"Globally, 22% of Kagglers use MySQL most often", hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df_country_bd = df_bdp1[df_bdp1.iloc[:,3]==country][df_bdp1.columns[173]].value_counts(normalize=True)*100

country_avg = df_country_bd.loc['MySQL ']

if country_avg > 22:
    title=f"In {country}, this percentage increases to {country_avg:.0f}%"
else:
    title=f"In {country}, this percentage decreases to {country_avg:.0f}%"
    
fig = px.pie(df_country_bd, df_country_bd.index, df_country_bd.values, title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

categories = sorted(list(df_bdp1.iloc[:,173].unique()))
df_country_bd = df_bdp1[df_bdp1.country_agg==country].iloc[:,173].value_counts(normalize=True).sort_index()
df_other_bd = df_platform[df_platform.country_agg=='Others'].iloc[:,173].value_counts(normalize=True).sort_index()

for index in df_other_bd.index:
        if index not in df_country_bd.index:
            df_country_bd[index] = 0
df_country_bd.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, x=categories, y=df_country_bd.values*100),
    go.Bar(name='Others', x=categories, y=df_other_bd.values*100)
])
# Change the bar mode
fig.update_layout(
    title=f'Big Data Product preference of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage',
    xaxis={'categoryorder':'array',
           'categoryarray':categories}
)
fig.show()

## 21. Business Intelligence Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following business intelligence tools do you use on a regular basis?</b>'</font>

In [ ]:
bit_cols = [col for col in df.columns 
           if "Which of the following business intelligence tools do you use on a regular basis?" in col]
df_bit = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in bit_cols]
mapping_dict = dict(zip(bit_cols, mapper))
df_bit = df_bit[bit_cols + [df_bit.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_bit.dropna(how='all', subset=mapper, inplace=True)
df_bit.drop(columns=['None'], inplace=True)

fig = px.bar(df_bit[df_bit.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'27% of all respondents use Tableau',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_bit[df_bit.iloc[:,-1]==country][df_bit.columns[:-2]].count().sort_values(ascending=False)
pop_bit = country_most_pop.index[0].strip()
if '(' in pop_bit:
    pop_bit = pop_bit.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_bit[df_bit.iloc[:,-1]==country])
if pop_bit == 'Tableau':
    if most_pop_pct > 27:
        title = f'{pop_bit} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop_bit} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop_bit} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_bit[df_bit.iloc[:,-1]==country][df_bit.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

df_bit1 = df[(~df.iloc[:,189].isna()) & (df.iloc[:,189]!='None')]

fig = px.pie(df_bit1, df_bit1.columns[189], 
             title=f"Globally, 36% of Kagglers use Tableau most often", hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df_country_bi = df_bit1[df_bit1.iloc[:,3]==country][df_bit1.columns[189]].value_counts(normalize=True)*100

country_avg = df_country_bi.loc['Tableau']

if country_avg > 36:
    title=f"In {country}, this percentage increases to {country_avg:.0f}%"
else:
    title=f"In {country}, this percentage decreases to {country_avg:.0f}%"
    
fig = px.pie(df_country_bi, df_country_bi.index, df_country_bi.values, title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

categories = sorted(list(df_bit1.iloc[:,189].unique()))
df_country_bi = df_bit1[df_bit1.country_agg==country].iloc[:,189].value_counts(normalize=True).sort_index()
df_other_bi = df_bit1[df_bit1.country_agg=='Others'].iloc[:,189].value_counts(normalize=True).sort_index()

for index in df_other_bi.index:
    if index not in df_country_bi.index:
        df_country_bi[index] = 0
df_country_bi.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, x=categories, y=df_country_bi.values*100),
    go.Bar(name='Others', x=categories, y=df_other_bi.values*100)
])
# Change the bar mode
fig.update_layout(
    title=f'Business Intelligence Product preference of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage',
    xaxis={'categoryorder':'array',
           'categoryarray':categories}
)
fig.show()

## 22. ML Lifecycle Automation Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Do you use any automated machine learning tools (or partial AutoML tools) on a regular basis?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Do you use any automated machine learning tools (or partial AutoML tools) on a regular basis?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['No / None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'9% of all respondents use Automation of full ML pipelines',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
most_pop_pct = country_most_pop.loc['Automation of full ML pipelines (e.g. Google AutoML, H20 Driverless AI)']*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if most_pop_pct > 9:
    title = f'Automation of full ML pipelines is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f'Automation of full ML pipelines remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 23. AutoML Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following automated machine learning tools (or partial AutoML tools) do you use on a regular basis?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Which of the following automated machine learning tools (or partial AutoML tools) do you use on a regular basis?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['No / None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'29% of all respondents use Auto-Sklearn',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Auto-Sklearn':
    if most_pop_pct > 29:
        title = f'{pop} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 24. ML Experiment Management Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following automated machine learning tools (or partial AutoML tools) do you use on a regular basis?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Do you use any tools to help manage machine learning experiments?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['No / None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'20% of all respondents use TensorBoard',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'TensorBoard':
    if most_pop_pct > 20:
        title = f'{pop} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 25. Data Anaysis sharing/ML Experiment Deployment Platform
<font color=red>Only includes respondents who share their Data Analysis/ML Experiments publicly</font>

In [ ]:
cols = [col for col in df.columns 
           if "Where do you publicly share or deploy your data analysis or machine learning applications?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['I do not share my work publicly'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'52% of all respondents use share their work on GitHub',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'GitHub':
    if most_pop_pct > 52:
        title = f'{pop} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

# Learning Preferences
## 26. Data Science Learning Platform
<font color=red>Only includes respondents who chose atleast one option for the question '<b>On which platforms have you begun or completed data science courses?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "On which platforms have you begun or completed data science courses?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'53% of all respondents use Coursera for learning',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Coursera':
    if most_pop_pct > 53:
        title = f'{pop} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 27. Data Science Media Sources
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Who/what are your favorite media sources that report on data science topics?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Who/what are your favorite media sources that report on data science topics?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'60% of all respondents use Kaggle to stay updated on Data Science',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Kaggle':
    if most_pop_pct > 60:
        title = f'{pop} is even more popular in {country},<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
    else:
        title = f'{pop} remains the most popular in {country} too,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis'
else:
    title = f"However, in {country}, {pop} is popular,<br>with {most_pop_pct:.0f}% of respondents using it on a regular basis"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 28. Cloud Computing Platforms
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following cloud computing platforms do you hope to become more familiar with in the next 2 years?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Which of the following cloud computing platforms do you hope to become more familiar with in the next 2 years?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'59% of all respondents hope to become familiar with AWS',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Amazon Web Services':
    if most_pop_pct > 60:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 29. Cloud Computing Products
<font color=red>Only includes respondents who chose atleast one option for the question '<b>In the next 2 years, do you hope to become more familiar with any of these specific cloud computing products?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "In the next 2 years, do you hope to become more familiar with any of these specific cloud computing products?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'Inspite of AWS\'s popularity, more respondents globally want to become more familiar<br>Google Cloud Compute Engine (47%)',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Google Cloud Compute Engine':
    if most_pop_pct > 47:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 30. Machine Learning Products
<font color=red>Only includes respondents who chose atleast one option for the question '<b>In the next 2 years, do you hope to become more familiar with any of these specific machine learning products?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "In the next 2 years, do you hope to become more familiar with any of these specific machine learning products?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'Google maintains the lead here too, with 50% respondents globally<br>wanting to become more familiar with Google Cloud AI Platform/ML Engine',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Google Cloud AI Platform / Google Cloud ML Engine':
    if most_pop_pct > 50:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 31. Big Data Products
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you hope to become more familiar with in the next 2 years?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you hope to become more familiar with in the next 2 years?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'At 50%, the humble MySQL is still the most desired Big Data Product',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'MySQL':
    if most_pop_pct > 50:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 32. Business Intelligence Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which of the following business intelligence tools do you hope to become more familiar with in the next 2 years?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Which of the following business intelligence tools do you hope to become more familiar with in the next 2 years?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'47% of all respondents hope to become more familiar with Tableau',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Tableau':
    if most_pop_pct > 47:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 33. ML Workflow Automation Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which categories of automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Which categories of automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'43% of all respondents hope to become more familiar with Automated Model Selection',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Automated model selection':
    if most_pop_pct > 43:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br>although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 34. Machine Learning / Partial AutoML Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Which specific automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "Which specific automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'51% of all respondents hope to become more familiar with Auto-Sklearn',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'Auto-Sklearn':
    if most_pop_pct > 51:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 35. ML Experiment Management Tools
<font color=red>Only includes respondents who chose atleast one option for the question '<b>In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments?</b>'</font>

In [ ]:
cols = [col for col in df.columns 
           if "In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments?" in col]
df_filtered = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in cols]
mapping_dict = dict(zip(cols, mapper))
df_filtered = df_filtered[cols + [df_filtered.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_filtered.dropna(how='all', subset=mapper, inplace=True)
df_filtered.drop(columns=['None'], inplace=True)

fig = px.bar(df_filtered[df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'44% of all respondents hope to become more familiar with TensorBoard',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False)
pop = country_most_pop.index[0].strip()
if '(' in pop:
    pop = pop.split('(')[0].strip()
most_pop_pct = country_most_pop[0]*100/len(df_filtered[df_filtered.iloc[:,-1]==country])
if pop == 'TensorBoard':
    if most_pop_pct > 44:
        title = f'In {country}, at {most_pop_pct:.0f}%,<br>even more respondents hope to become familiar with {pop}'
    else:
        title = f'In {country}, {pop} is still highest in demand,<br> although slightly less popular at {most_pop_pct:.0f}%'
else:
    title = f"However, in {country}, more respondents hope to become familiar with {pop} ({most_pop_pct:.0f}%)"

fig = px.bar(df_filtered[df_filtered.iloc[:,-1]==country][df_filtered.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

# Workplace

## 36. Size
<font color=red>Only includes respondents who answered the question '<b>What is the size of the company where you are employed?</b>'</font>

In [ ]:
df_wp_size = df[~df.iloc[:,107].isna()]

fig = px.pie(df_wp_size, df_wp_size.columns[107], title='51% of all Kagglers work in companies with less than 250 employees', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

size_pct = len(df_wp_size[(df_wp_size[df_wp_size.columns[3]]==country) & (df_wp_size[df_wp_size.columns[107]].isin(['0-49 employees','50-249 employees']))])*100 \
/len(df_wp_size[df_wp_size[df_wp_size.columns[3]]==country])
if size_pct > 51:
    title = f"For {country}, this percentage increases to {size_pct:.0f}%<br>Kagglers from {country} tend to work in smaller companies"
elif size_pct < 51:
    title = f"For {country}, this percentage decreases to {size_pct:.0f}%<br>Kagglers from {country} tend to work in larger companies"
else:
    title = f"At {size_pct:.0f}%, it is the same for {country} too"
fig = px.pie(df_wp_size[df_wp_size[df_wp_size.columns[3]]==country], df_wp_size.columns[107], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df_country = df_wp_size[df_wp_size.country_agg==country][df_wp_size.columns[107]].value_counts(normalize=True).sort_index()
df_others = df_wp_size[df_wp_size.country_agg=='Others'][df_wp_size.columns[107]].value_counts(normalize=True).sort_index()

for index in df_others.index:
        if index not in df_country.index:
            df_country[index] = 0
df_country.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, y=df_country.values*100),
    go.Bar(name='Others', y=df_others.values*100)
])

# Change the bar mode
fig.update_layout(
    barmode='group',
    title=f'Workplace size of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage of respondents',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(5)],
        ticktext = df_wp_size[df_wp_size.columns[107]].sort_values().unique()
    )
)

fig.show()

## 37. Data Science Team Size
<font color=red>Only includes respondents who answered the question '<b>Approximately how many individuals are responsible for data science workloads at your place of business?</b>'</font>

In [ ]:
df_dsts_size = df[~df.iloc[:,108].isna()]

fig = px.pie(df_dsts_size, df_dsts_size.columns[108], 
             title='60% of all Kagglers work in companies with less than 5 individuals handling Data Science workloads', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()


size_pct = len(df_dsts_size[(df_dsts_size[df_dsts_size.columns[3]]==country) & (df_dsts_size[df_wp_size.columns[108]].isin(['0','1-2', '3-4']))])*100 \
/len(df_dsts_size[df_dsts_size[df_dsts_size.columns[3]]==country])
if size_pct > 60:
    title = f"For {country}, this percentage increases to {size_pct:.0f}%<br>Kagglers from {country} work in companies with smaller Data Science teams"
elif size_pct < 60:
    title = f"For {country}, this percentage decreases to {size_pct:.0f}%<br>Kagglers from {country} work in companies with larger Data Science teams"
else:
    title = f"At {size_pct:.0f}%, it is the same for {country} too"
fig = px.pie(df_dsts_size[df_dsts_size[df_dsts_size.columns[3]]==country], df_dsts_size.columns[108], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df_country = df_dsts_size[df_dsts_size.country_agg==country][df_dsts_size.columns[108]].value_counts(normalize=True).sort_index()
df_others = df_dsts_size[df_dsts_size.country_agg=='Others'][df_dsts_size.columns[108]].value_counts(normalize=True).sort_index()

for index in df_others.index:
        if index not in df_country.index:
            df_country[index] = 0
df_country.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, y=df_country.values*100),
    go.Bar(name='Others', y=df_others.values*100)
])

# Change the bar mode
fig.update_layout(
    barmode='group',
    title=f'Company Data Science Team size of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage of respondents',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(7)],
        ticktext = df_dsts_size[df_dsts_size.columns[108]].sort_values().unique()
    )
)

fig.show()

## 38. Machine Learning Adoption at Work
<font color=red>Only includes respondents who answered the question '<b>Does your current employer incorporate machine learning methods into their business?</b>'</font>

In [ ]:
df_mla = df[~df.iloc[:,109].isna()]

fig = px.pie(df_mla, df_mla.columns[109], 
             title='20% of all Kagglers work in companies which don\'t use ML models', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

size_pct = len(df_mla[(df_mla[df_mla.columns[3]]==country) & 
                      (df_mla[df_mla.columns[109]].isin(['No (we do not use ML methods)']))])*100 \
/len(df_mla[df_mla[df_mla.columns[3]]==country])
if size_pct > 20:
    title = f"For {country}, this percentage increases to {size_pct:.0f}%<br>Less Kagglers from {country} work in a company using ML models"
elif size_pct < 20:
    title = f"For {country}, this percentage decreases to {size_pct:.0f}%<br>More Kagglers from {country} work in a company using ML models"
else:
    title = f"At {size_pct:.0f}%, it is the same for {country} too"
fig = px.pie(df_mla[df_mla[df_mla.columns[3]]==country], df_mla.columns[109], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

df_country = df_mla[df_mla.country_agg==country][df_mla.columns[109]].value_counts(normalize=True).sort_index()
df_others = df_mla[df_mla.country_agg=='Others'][df_mla.columns[109]].value_counts(normalize=True).sort_index()

for index in df_others.index:
        if index not in df_country.index:
            df_country[index] = 0
df_country.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, y=df_country.values*100),
    go.Bar(name='Others', y=df_others.values*100)
])

# Change the bar mode
fig.update_layout(
    barmode='group',
    title=f'ML adoption in workplaces of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage of respondents',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(7)],
        ticktext = df_mla[df_mla.columns[109]].sort_values().unique()
    )
)

fig.show()

## 39. Role at Work
<font color=red>Only includes respondents who chose atleast one option for the question '<b>Select any activities that make up an important part of your role at work</b>'</font>

In [ ]:
role_cols = [col for col in df.columns 
           if "Select any activities that make up an important part of your role at work" in col]
df_role = df.copy()
mapper = [col.split('- ',maxsplit=2)[2] for col in role_cols]
mapping_dict = dict(zip(role_cols,mapper))
df_role = df_role[role_cols + [df.columns[3]] + ['country_agg']].rename(columns=mapping_dict)
df_role.dropna(how='all', subset=mapper, inplace=True)

fig = px.bar(df_role[df_role.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=f'21% of all respondents are involved in ML research',
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

country_most_pop = df_role[df_role.iloc[:,-1]==country][df_role.columns[:-2]].count().sort_values(ascending=False)
research_cnt = country_most_pop.loc['Do research that advances the state of the art of machine learning']
research_pct = research_cnt*100/len(df_role[df_role.iloc[:,-1]==country])
if research_pct > 21:
    title = f'At {research_pct:.0f}%, more Kagglers from {country} are involved in ML research than the global average'
elif research_pct < 21:
    title = f'At {research_pct:.0f}%, less Kagglers from {country} are involved in ML research than the global average'
else:
    title = f'At {research_pct:.0f}%, the percentage of Kagglers {country} involved in ML research is the same as the global average'

fig = px.bar(df_role[df_role.iloc[:,-1]==country][df_role.columns[:-2]].count().sort_values(ascending=False))
fig.update_layout(
    title=title,
    xaxis_title=None,
    yaxis_title='Number of respondents',
    showlegend=False
)
fig.show()

## 40. Spend on ML/Cloud
<font color=red>Only includes respondents who answered the question '<b>Approximately how much money have you (or your team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (approximate $USD)?</b>'</font>

In [ ]:
df_ml_spend = df[~df.iloc[:,119].isna()]
df_ml_spend[df_ml_spend.columns[119]] = df_ml_spend[df_ml_spend.columns[119]].apply(lambda x: x.replace('$','').replace(',',''))

fig = px.pie(df_ml_spend, df_ml_spend.columns[119], 
             title='37% of all Kagglers work in companies which don\'t spend on ML/Cloud', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

size_pct = len(df_ml_spend[(df_ml_spend.iloc[:,3]==country) & (df_ml_spend.iloc[:,119]=='0 (USD)')])*100/sum(df_ml_spend.iloc[:,3]==country)

if size_pct > 37:
    title = f"For {country}, this percentage increases to {size_pct:.0f}%<br>More Kagglers from {country} work in a company not spending on ML/Cloud"
elif size_pct < 37:
    title = f"For {country}, this percentage decreases to {size_pct:.0f}%<br>More Kagglers from {country} work in a company spending on ML/Cloud"
else:
    title = f"At {size_pct:.0f}%, it is the same for {country} too"
    
fig = px.pie(df_ml_spend[df_ml_spend[df_ml_spend.columns[3]]==country], df_ml_spend.columns[119], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

df_country = df_ml_spend[df_ml_spend.country_agg==country][df_ml_spend.columns[119]].value_counts(normalize=True).sort_index()
df_others = df_ml_spend[df_ml_spend.country_agg=='Others'][df_ml_spend.columns[119]].value_counts(normalize=True).sort_index()
for index in df_others.index:
        if index not in df_country.index:
            df_country[index] = 0
df_country.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, y=df_country.values*100),
    go.Bar(name='Others', y=df_others.values*100)
])

# Change the bar mode
fig.update_layout(
    barmode='group',
    title=f'Spend on ML/Cloud in workplaces of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage of respondents',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(6)],
        ticktext = df_ml_spend[df_ml_spend.columns[119]].sort_values().unique()
    )
)

fig.show()

## 41. Primary Tool used at Work
<font color=red>Only includes respondents who answered the question '<b>What is the primary tool that you use at work or school to analyze data?</b>'</font>

In [ ]:
df_mla = df[~df.iloc[:,243].isna()]

fig = px.pie(df_mla, df_mla.columns[243], 
             title='46% of all Kagglers primarily use their local development environment', hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

size_pct = len(df_mla[(df_mla[df_mla.columns[3]]==country) & 
                      (df_mla[df_mla.columns[243]].isin(['Local development environments (RStudio, JupyterLab, etc.)']))])*100 \
/len(df_mla[df_mla[df_mla.columns[3]]==country])
if size_pct > 46:
    title = f"For {country}, this percentage increases to {size_pct:.0f}%"
elif size_pct < 46:
    title = f"For {country}, this percentage decreases to {size_pct:.0f}%"
else:
    title = f"At {size_pct:.0f}%, it is the same for {country} too"
fig = px.pie(df_mla[df_mla[df_mla.columns[3]]==country], df_mla.columns[243], title=title, hole=0.6)
fig.update_traces(textinfo='percent+label')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide', showlegend=False)
fig.show()

df_country = df_mla[df_mla.country_agg==country][df_mla.columns[243]].value_counts(normalize=True).sort_index()
df_others = df_mla[df_mla.country_agg=='Others'][df_mla.columns[243]].value_counts(normalize=True).sort_index()

for index in df_others.index:
        if index not in df_country.index:
            df_country[index] = 0
df_country.sort_index(inplace=True)

fig = go.Figure(data=[
    go.Bar(name=country, y=df_country.values*100),
    go.Bar(name='Others', y=df_others.values*100)
])

# Change the bar mode
fig.update_layout(
    barmode='group',
    title=f'Primary Tool used at Work in workplaces of Kagglers from {country} compared to others',
    xaxis_title=None,
    yaxis_title='Percentage of respondents',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(7)],
        ticktext = df_mla[df_mla.columns[243]].sort_values().unique()
    )
)

fig.show()

# Thanks!

That should have taken a while. Thanks for going through my notebook.  
Please don't forget to <font color=green><b>UPVOTE</b></font>, and suggest what else you would like to be added.  
Also, would love to know which analysis you found particularly interesting, and why.

## Check-out my similar works on the Kaggle 2020 survey
* [Kaggle 2020: India VS the World](https://www.kaggle.com/siddhantsadangi/kaggle-2020-india-vs-the-world-all-questions)
* [Kaggle 2020: USA VS the World](https://www.kaggle.com/siddhantsadangi/kaggle-2020-usa-vs-the-world-all-questions)